<h3>Description du jeu</h3>

<h5>ELEMENTS</h5>
En fonction du jeu : location du passager ou de la destination:

- bleu

- rouge

- jaune

- vert

Autres lieux: lieux divers


<h5>ACTIONS</h5>

- 0: déplacer vers le Sud (bas)
- 1: déplacer vers le Nord (haut)
- 2: déplacer vers l'Est (gauche) 
- 3: déplacer vers le Ouest (droite) 
- 4: prendre passager
- 5: déposer passager


<h5>RECOMPENSES</h5> 

- DONE (fin de jeu) : 20

- Déposer le passager à la mauvaise destination : -10

- Prendre un autre passager que le passager bleu : -10

- Pour toute autre action, parmi les actions de 0 -> 6 : recompense -1


<h5>REGLES</h5>

- Le taxi se déplace selon les actions indiquées.

- On ne peut pas se déplacer à travers un mur.

- Le jeu est fini quand le bon passager est déposé à la bonne déstination.

Plus de détail en anglais : 
https://www.gymlibrary.dev/environments/toy_text/taxi/

<h3>Indications sur le code</h3>
Choisir une nouvelle action de manière aléatoire : 

    `action = env.action_space.sample()`
    
Choisir la meilleure action pour un état donné `s`: 

    Choisir la plus grande valeur pour `Q[s]`

Prendre une action avec gym :

    `obs, reward, done, truncated, info = env.step(action)`
 
 `obs`: nouvel état, 
 `reward`: recompense obtenue, 
 `done`: boolean, indiquant si le jeu est fini, 
 `truncated`: boolean, indique si le jeu est fini suite à une action irrégulière (pas applicable dans ce cas).
 `info`, diverses infos sur l'action

Choisir la plus grande valeur d'un tableau T: 

    `max_val = np.argmax(T)`

In [1]:
import gym
import numpy as np

env = gym.make('Taxi-v3')
env.reset()

NUM_ACTIONS = env.action_space.n # nombre des actions 
NUM_STATES = env.observation_space.n # nombre des états

# Tablea dans lequel on va stocker les valeurs de l'action; dimension NUM_STATES * NUM_ACTIONS
Q = np.zeros([NUM_STATES, NUM_ACTIONS]) 

gamma = 0.95 # facteur d'actualisation (discount factor)
alpha = 0.1 # taux d'apprentissage (learning rate)
epsilon = 0.1 # facteur pour epsilon greedy (approche glutonne)

<h3>Entraînement du modèle</h3>  
<br/>
Pour l'entraînement on doit remplir le tableau Q, qui où on a une valeur pour chaque état et chaque action correspondante.
<br/><br/>
Dans l'étape d'entraînement, on joue le jeu plusieurs fois (boucle for), et à chaque jeu on passe d'une état à un autre (boucle while). Dans la boucle while, on a le couple [etat - action] courants et [etat - action] suivants. Avec ces éléments et la recompense, on met à jour, au fur et à mesure, le modèle, notamment les cases correspondantes dans le tableau Q, avec l'algorithme QLearning (voir cours 3).
<br/><br/>
1. On a besoin donc des couples :
<br/>
(état courant - action courante) : (s, a)
<br/>
A partir d'un état `s`, on choisit l'action `a` avec la politique epsilon greedy 
<br/>
<br/>
2. (état suivant - action suivante) : (s_, a_)
<br/>
<br/>
A partir de l'état `s`, en prenant l'action `a`, on arrive dans l'état suivant `s_`. Il nous faut maintenant l'action `a_` correspondante. Avec l'algorithme QLearning, cette action est la meilleure action de l'état `s_`. On la trouve dans le tableau Q ou pour chaque état et pour chaque action on a des valeurs (on prend la valeur la plus élevée pour l'état `s_`).

In [2]:
for episode in range(1,500001):
    # boolean qui indique si on a fini le jeu on non : 
    # False si on n'a pas fini le jeu
    # True si on a  fini le jeu
    done = False 
    obs, _ = env.reset() 
    c = 0
    rng = np.random.default_rng() # Genere un nombre random entre zéro et 1
    while done != True: # tant qu'on n'a pas fini notre jeu :
        # on choisit l'action avec la méthode epsilon greedy (approche glutonne) :
        # on choisit une action aléatoire pour une probabilité epsilon des cas et 
        # on choisit la meilleure action pour (1 - epsion) cas
        
        # ici, on va générer un numéro aléatoire entre 0 et 1 : 
        random_num = rng.random()
        if random_num < epsilon: # exploration : on choisit une action aléatoire
            action = env.action_space.sample() # genere une action aléatoire
        else: # exploitation : on choisit la meilleure action pour l'état présent : obs
            action = np.argmax(Q[obs])  # Parmis les actions effectuable on prend la meilleur
            
        # On prend une action :     
        obs2, rew, done, truncated, info = env.step(action)  # Execute l'action qu'on a choisit avec l'epsilon greedy
        
        # On met à jour la matrice Q(s, a) avec la formule Q-learning (voir cours) :
        Q[obs,action] += alpha * (rew + gamma * (Q[obs2, np.argmax(Q[obs2])] - Q[obs,action]))
 # A COMPLETER !!!
        
        # On se positionne sur le nouvel état:
        obs = obs2   
        
    if episode % 5000 == 0:
        #à chaque 5000 pas, testons 100 jeux afin d'obtenir la moyenne des retours et 
        # vérifier si l'apprentissage automatique est fini
        
        rew_average = 0.
        for i in range(100):
            obs, _ = env.reset()
            done=False
            while done != True: 
                action = np.argmax(Q[obs]) # on choisit l'action maximum pour l'état obs:
                obs, rew, done, truncated, info = env.step(action) # prend l'action séléctionée
                if done is True and rew == 20:
                    rew_average += 1
        rew_average=rew_average/100
        print('Episode {} a la recompense moyenne: {}'.format(episode,rew_average))
        
        if rew_average > 0.8: 
            # On défini le jeu FrozenLake-v0 fini 
            # si on a la recompense moyenne de 80 sur 100 essais
            print("Taxi a été résolu !")
            break

Episode 5000 a la recompense moyenne: 1.0
Taxi a été résolu !


In [3]:
# On initialise le jeu avec les paramètres qui permettent d'afficher un jeu:
env = gym.make('Taxi-v3', render_mode="human")

# Voyons maintenant comment notre problème est résolu, après avoir amélioré la politique :
G = 0.
obs, _ = env.reset()
done=False
while done != True: 
    # On choisit la meilleure action pour l'état:
    action = np.argmax(Q[obs])
    
    # On prend cette action :
    obs, rew, done, truncated, info = env.step(action)
    
    # On ajoute la recompense au retour (recompense totale de l'episode)
    G += rew
    
# On affiche les étapes du jeu : 
env.render()

# On affiche la recompense:
print("Récompense:", G)



Récompense: 6.0


: 